In [1]:
import torch
from torch import nn
from d2l import torch as d2l
import os # 解决库冲突
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [2]:
def dropout_layer(X, dropout):
    assert 0 <= dropout <= 1
    # assert: 断言是 Python 中的调试工具之一，通常用于检查代码中的某些条件。
    # 如果条件为 True，程序继续执行；如果为 False，则会抛出一个 AssertionError，并终止程序。
    if dropout == 1:
        return torch.zeros_like(X)
    mask = (torch.rand(X.shape) > dropout).float() # 这里的随机有点特别
    return mask * X / (1.0 - dropout)

In [3]:
X = torch.arange(16, dtype = torch.float32).reshape((2, 8))
print('dropout_p = 0:', dropout_layer(X, 0))
print('dropout_p = 0.5:', dropout_layer(X, 0.5))
print('dropout_p = 1:', dropout_layer(X, 1))

dropout_p = 0: tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11., 12., 13., 14., 15.]])
dropout_p = 0.5: tensor([[ 0.,  2.,  4.,  0.,  8., 10., 12., 14.],
        [16., 18., 20.,  0., 24., 26.,  0.,  0.]])
dropout_p = 1: tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])


In [4]:
class DropoutMLPScratch(d2l.Classifier):
    def __init__(self, num_outputs, num_hiddens_1, num_hiddens_2, dropout_1, dropout_2, lr):
        super().__init__()
        self.save_hyperparameters()
        self.lin1 = nn.LazyLinear(num_hiddens_1)
        self.lin2 = nn.LazyLinear(num_hiddens_2)
        self.lin3 = nn.LazyLinear(num_outputs)
        self.relu = nn.ReLU()

    def forward(self, X):
        H1 = self.relu(self.lin1(X.reshape((X.shape[0], -1)))) # 内部展平
        if self.training: # 如果在训练模式下，应用 dropout
            H1 = dropout_layer(H1, self.dropout_1)
        H2 = self.relu(self.lin2(H1))
        if self.training:
            H2 = dropout_layer(H2, self.dropout_2)
        return self.lin3(H2)

In [5]:
hparams = {'num_outputs':10, 'num_hiddens_1':256, 'num_hiddens_2':256,
           'dropout_1':0.5, 'dropout_2':0.5, 'lr':0.1}
model = DropoutMLPScratch(**hparams)
data = d2l.FashionMNIST(batch_size=256)
trainer = d2l.Trainer(max_epochs=10)
trainer.fit(model, data)

In [6]:
# 下面开始进行简洁实现
class DropoutMLP(d2l.Classifier):
    def __init__(self, num_outputs, num_hiddens_1, num_hiddens_2,
                 dropout_1, dropout_2, lr):
        super().__init__()
        self.save_hyperparameters()
        self.net = nn.Sequential( # 这玩意实在太方便了！！
            nn.Flatten(), nn.LazyLinear(num_hiddens_1), nn.ReLU(),
            nn.Dropout(dropout_1), nn.LazyLinear(num_hiddens_2), nn.ReLU(),
            nn.Dropout(dropout_2), nn.LazyLinear(num_outputs))

In [7]:
model = DropoutMLP(**hparams)
trainer.fit(model, data)

""" Exercise
1. 说实话，我没试出来有什么太大区别，但是确实最好输入层一侧调小一点，训练中可以调大一点。我看评论区推荐 0.2-0.5 的范围，感觉这玩意没什么理论呢。
2. 增加 epoch 数量会使模型更充分地学习训练数据。使用 dropout 通常可以防止过拟合，因此在更多 epoch 后仍能保持较好的泛化性能。
3. 使用 dropout 会增加激活值的方差，因为部分神经元的输出会随机置零。这会导致每一层的激活值分布更加分散。在不使用 dropout 的情况下，激活值的方差会相对稳定。
4. Dropout 主要用于训练阶段以防止过拟合。在测试时，我们希望获得确定的预测，因此不再随机丢弃神经元。测试阶段的模型应该反映模型学习到的所有特征，而不是随机抑制一部分特征。
5. Dropout 和权重衰减都用于正则化，防止过拟合。权重衰减使得权重值趋向于零，而 dropout 则随机丢弃部分神经元的输出。效果是否叠加取决于数据和模型。如果正则化太强，可能会导致模型欠拟合。
6. 如果将 dropout 应用到权重矩阵的个别权重而不是激活值，可能会影响到参数的更新方式。这种方法可能会导致模型的训练变得不稳定，因为某些权重在某次迭代中没有更新。
7. 一种新的方法可以是在每一层加入高斯噪声，代替 dropout。例如，可以在前向传播时加入少量的正态分布噪声，使得模型在输入数据发生轻微变化时更加鲁棒。
""""